In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import mean_squared_error
import numpy as np
import warnings
# Suppress only the SettingWithCopyWarning
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)
path = "proj1/fold{}/{}.csv"

In [60]:
df = pd.read_csv(path.format(1, "train"))
df.replace('',None).isna().sum().sort_values()

PID                  0
MS_SubClass          0
MS_Zoning            0
Lot_Frontage         0
Lot_Area             0
                  ... 
Sale_Condition       0
Longitude            0
Garage_Yr_Blt      108
Mas_Vnr_Type      1233
Misc_Feature      1975
Length: 83, dtype: int64

In [5]:
path = "proj1/fold{}/{}.csv"

data = {}
#X_train = np.array(10, 2051, 83)
#X_test =  np.array(10, 879, 82)
#Y = np.array(10, 879, 2
trains = []
tests = []
for fold in range(1,11):
    train = pd.read_csv(path.format(fold, "train"))
    test = pd.read_csv(path.format(fold, "test"))
    test_y = pd.read_csv(path.format(fold, "test_y"))
    trains.append(train)
    tests.append(test)
    

    #nulls = train.isna().sum()
    #print(nulls[nulls > 0])
    #features_keep = list(abs(train.corrwith(train.Sale_Price)).sort_values(ascending=False)[:-100].index)
    #print(features_keep)
    #plt.figure(figsize=(8,6))
    #sns.distplot(train["Sale_Price"])
    #title = plt.title("House Price Distribution")
    #train_cols = pd.get_dummies(train, drop_first=True)
    #test_cols = pd.get_dummies(test, drop_first=True)

all_train = pd.concat(trains)
all_test = pd.concat(tests)


all_train['Total_SF'] = all_train[['First_Flr_SF','Second_Flr_SF','Total_Bsmt_SF']].copy().sum(axis=1)
all_train['Total_Bath'] = all_train.loc[:,'Full_Bath'] + (0.5 * all_train.loc[:,'Half_Bath'])
all_train['Has_Pool'] = all_train['Pool_QC'].map({"No_Pool":0, 'Excellent':1, 'Good':1, 'Typical':0, 'Fair':0})

corrs = all_train.select_dtypes(include='number').corr().Sale_Price.abs().sort_values()
corrs[corrs > 0.1]

Has_Pool          0.111545
Kitchen_AbvGr     0.121616
Screen_Porch      0.126030
Enclosed_Porch    0.136201
BsmtFin_SF_1      0.142277
Bedroom_AbvGr     0.148072
Bsmt_Unf_SF       0.176934
Lot_Frontage      0.209532
Longitude         0.248249
PID               0.252680
Second_Flr_SF     0.262596
Lot_Area          0.283330
Half_Bath         0.283937
Bsmt_Full_Bath    0.285094
Latitude          0.296598
Open_Porch_SF     0.317107
Wood_Deck_SF      0.334206
Fireplaces        0.480482
TotRms_AbvGrd     0.503554
Mas_Vnr_Area      0.524119
Garage_Yr_Blt     0.524968
Year_Remod_Add    0.531350
Full_Bath         0.549344
Year_Built        0.558975
Total_Bath        0.584025
First_Flr_SF      0.643322
Garage_Area       0.644519
Garage_Cars       0.651167
Total_Bsmt_SF     0.652654
Gr_Liv_Area       0.716609
Total_SF          0.810819
Sale_Price        1.000000
Name: Sale_Price, dtype: float64

In [7]:
train[[col for col in train.columns if 'kitchen' in col.lower()]]


,Kitchen_AbvGr,Kitchen_Qual
0,1,Typical
1,1,Good
2,1,Typical
3,1,Good
4,1,Good
...,...,...
2046,2,Typical
2047,1,Typical
2048,1,Typical
2049,1,Typical


In [15]:
#train[[col for col in train.columns if 'pool' in col.lower()]]
train['Pool_QC'].unique()
train['Pool_QC'].map({"No_Pool":0, 'Excellent':1, 'Good':1, 'Typical':1, 'Fair':1})

0       0
1       0
2       0
3       0
4       0
       ..
2046    0
2047    0
2048    0
2049    0
2050    0
Name: Pool_QC, Length: 2051, dtype: int64

In [7]:
#train.select_dtypes(include='number').corr().Sale_Price.sort_values(ascending=False)

train.select_dtypes(include='number').corr().Sale_Price.abs().sort_values(ascending=False)

Sale_Price            1.000000
Gr_Liv_Area           0.712708
Garage_Cars           0.656239
Garage_Area           0.647097
Total_Bsmt_SF         0.639975
First_Flr_SF          0.638684
Year_Built            0.562816
Full_Bath             0.544902
Year_Remod_Add        0.534348
Garage_Yr_Blt         0.530769
Mas_Vnr_Area          0.525480
TotRms_AbvGrd         0.496110
Fireplaces            0.470213
Wood_Deck_SF          0.337751
Open_Porch_SF         0.315202
Latitude              0.298313
Lot_Area              0.296368
Bsmt_Full_Bath        0.290153
Half_Bath             0.283431
PID                   0.254454
Second_Flr_SF         0.253695
Longitude             0.237673
Lot_Frontage          0.200385
Bsmt_Unf_SF           0.186392
Enclosed_Porch        0.144179
Bedroom_AbvGr         0.140608
BsmtFin_SF_1          0.138790
Kitchen_AbvGr         0.119098
Screen_Porch          0.117423
Pool_Area             0.080572
Bsmt_Half_Bath        0.039123
Three_season_porch    0.031319
Year_Sol

In [17]:
train.select_dtypes(include='object')

,MS_SubClass,MS_Zoning,Street,Alley,Lot_Shape,Land_Contour,Utilities,Lot_Config,Land_Slope,Neighborhood,...,Garage_Type,Garage_Finish,Garage_Qual,Garage_Cond,Paved_Drive,Pool_QC,Fence,Misc_Feature,Sale_Type,Sale_Condition
0,One_Story_1946_and_Newer_All_Styles,Residential_High_Density,Pave,No_Alley_Access,Regular,Lvl,AllPub,Inside,Gtl,North_Ames,...,Attchd,Unf,Typical,Typical,Paved,No_Pool,Minimum_Privacy,NaN,WD,Normal
1,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Corner,Gtl,North_Ames,...,Attchd,Unf,Typical,Typical,Paved,No_Pool,No_Fence,Gar2,WD,Normal
2,Two_Story_1946_and_Newer,Residential_Low_Density,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Inside,Gtl,Gilbert,...,Attchd,Fin,Typical,Typical,Paved,No_Pool,Minimum_Privacy,NaN,WD,Normal
3,Two_Story_1946_and_Newer,Residential_Low_Density,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Inside,Gtl,Gilbert,...,Attchd,Fin,Typical,Typical,Paved,No_Pool,No_Fence,NaN,WD,Normal
4,One_Story_PUD_1946_and_Newer,Residential_Low_Density,Pave,No_Alley_Access,Regular,Lvl,AllPub,Inside,Gtl,Stone_Brook,...,Attchd,Fin,Typical,Typical,Paved,No_Pool,No_Fence,NaN,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046,Split_or_Multilevel,Residential_Low_Density,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,CulDSac,Gtl,Mitchell,...,Detchd,Unf,Typical,Typical,Paved,No_Pool,Good_Privacy,NaN,WD,Normal
2047,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,Pave,No_Alley_Access,Slightly_Irregular,Low,AllPub,Inside,Mod,Mitchell,...,Attchd,Unf,Typical,Typical,Paved,No_Pool,Minimum_Privacy,NaN,WD,Normal
2048,Split_Foyer,Residential_Low_Density,Pave,No_Alley_Access,Regular,Lvl,AllPub,Inside,Gtl,Mitchell,...,No_Garage,No_Garage,No_Garage,No_Garage,Paved,No_Pool,Minimum_Privacy,Shed,WD,Normal
2049,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,Pave,No_Alley_Access,Regular,Lvl,AllPub,Inside,Mod,Mitchell,...,Attchd,RFn,Typical,Typical,Paved,No_Pool,No_Fence,NaN,WD,Normal


In [8]:
test.head()

,PID,MS_SubClass,MS_Zoning,Lot_Frontage,Lot_Area,Street,Alley,Lot_Shape,Land_Contour,Utilities,...,Pool_QC,Fence,Misc_Feature,Misc_Val,Mo_Sold,Year_Sold,Sale_Type,Sale_Condition,Longitude,Latitude
0,908276150,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,0,8926,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,...,No_Pool,Minimum_Privacy,NaN,0,10,2009,COD,Abnorml,-93.663302,42.018592
1,903451090,Two_Story_1945_and_Older,Residential_Medium_Density,57,6876,Pave,No_Alley_Access,Regular,Lvl,AllPub,...,No_Pool,Minimum_Privacy,NaN,0,8,2009,WD,Normal,-93.628299,42.025306
2,527110080,Two_Story_1946_and_Newer,Residential_Low_Density,0,13869,Pave,No_Alley_Access,Moderately_Irregular,Lvl,AllPub,...,No_Pool,No_Fence,NaN,0,7,2007,WD,Normal,-93.636146,42.061664
3,535457010,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,87,10000,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,...,No_Pool,No_Fence,NaN,0,2,2010,WD,Normal,-93.605943,42.034748
4,903484020,One_and_Half_Story_Finished_All_Ages,Residential_Medium_Density,60,10320,Pave,Gravel,Regular,Lvl,AllPub,...,No_Pool,Minimum_Privacy,NaN,0,6,2008,WD,Normal,-93.624580,42.023716


In [8]:
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import mean_squared_error
import numpy as np

def clean_and_process_categorical_features(train, test):

    original_columns = train.columns
    # impute missing categories with the mode
    for col in original_columns:
        train.loc[:, col] = train[col].fillna(train[col].mode()[0])
        test.loc[:, col] = test[col].fillna(test[col].mode()[0])

    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    encoder.fit(train)
    
    encoded_train = encoder.transform(train)
    encoded_test = encoder.transform(test)
    
    encoded_train_df = pd.DataFrame(encoded_train, columns=encoder.get_feature_names_out(original_columns))
    encoded_test_df = pd.DataFrame(encoded_test, columns=encoder.get_feature_names_out(original_columns))
    return encoded_train_df, encoded_test_df

def clean_and_process_numeric_features(data):
    # impute missing values with median
    for col in data.columns:
        data.loc[:, col] = data[col].fillna(data[col].median())

    cols = data.columns
    
    # standardize the features
    scaler = StandardScaler()
    data = scaler.fit_transform(data)
    return pd.DataFrame(data, columns = cols)

def add_feature_engineered_cols(data):
    
    # add some features
    data.loc[:, 'Total_SF'] = data[['First_Flr_SF','Second_Flr_SF','Total_Bsmt_SF']].copy().sum(axis=1)
    data.loc[:, 'Total_Bath'] = data.loc[:,'Full_Bath'] + (0.5 * data.loc[:,'Half_Bath'])
    data['Has_Pool'] = data['Pool_QC'].map({"No_Pool":0, 'Excellent':1, 'Good':1, 'Typical':0, 'Fair':0})
    data['Has_Quality_Kitchen'] = data['Kitchen_Qual'].map({'Excellent':1, 'Good':1, 'Typical':0, 'Fair':0, 'Poor':0})
    data['Has_Quality_Basement'] = data['Bsmt_Qual'].map({'Excellent':1, 'Good':1, 'Typical':0, 'Fair':0, 'Poor':0})

    # drop some of the cols
    data = data.drop(columns=['Bsmt_Qual', 'Kitchen_Qual', 'Pool_QC'])

    return data
    
def select_features_lasso(X_train, y_train, alpha=0.1):
    lasso_model = Lasso(alpha=alpha)
    lasso_model.fit(X_train, y_train)
    
    important_features_mask = lasso_model.coef_ != 0
    return important_features_mask

def fit_ridge(X_train, y_train, X_test, alpha=1.0):

    ridge_model = Ridge(alpha=alpha)
    ridge_model.fit(X_train, y_train)
    
    return ridge_model.predict(X_test)


for fold in range(1,11):
    train = pd.read_csv(path.format(fold, "train"))
    test = pd.read_csv(path.format(fold, "test"))
    test_y = pd.read_csv(path.format(fold, "test_y"))

    
    X_train = train.drop(columns=["Sale_Price", "PID"])
    y_train = train.Sale_Price.apply(np.log)

    X_test = test.drop(columns=["PID"])
    y_test = test_y.Sale_Price.apply(np.log)

    # remove features that have high number of null data confirm with
    # nulls = data.isna().sum()
    # print(data[data > 0])
    cols_to_drop = ['BsmtFin_SF_2', 'Misc_Val', 'Mo_Sold', 'Three_season_porch',
       'Year_Sold', 'Bsmt_Half_Bath', 'Low_Qual_Fin_SF', 'Pool_Area']

    
    #X_train = X_train.drop(columns=["Mas_Vnr_Type", "Garage_Yr_Blt", "Misc_Feature"])
    #X_test = X_test.drop(columns=["Mas_Vnr_Type", "Garage_Yr_Blt", "Misc_Feature"])
    X_train = X_train.drop(columns=cols_to_drop)
    X_test = X_test.drop(columns=cols_to_drop)

    X_train = add_feature_engineered_cols(X_train)
    X_test = add_feature_engineered_cols(X_test)
    
    numeric_cols = X_train.select_dtypes(include=[np.number]).columns
    categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns

    X_train_numeric = clean_and_process_numeric_features(X_train[numeric_cols])
    X_test_numeric = clean_and_process_numeric_features(X_test[numeric_cols])

    X_train_cat, X_test_cat = clean_and_process_categorical_features(X_train[categorical_cols], X_test[categorical_cols])

    X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
    X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

    assert all(X_train.columns==X_test.columns)
    important_features_mask = select_features_lasso(X_train, y_train, alpha=0.1)
    #print(important_features_mask)
    #print([x[1] for x in zip(important_features_mask, train_columns) if x[0]])
    #print(list(zip(important_features_mask, train_columns)))

    # Step 3: Train Ridge model using the selected feature
    #X_train = X_train[:, important_features_mask]
    #X_test = X_test[:, important_features_mask]

    #print(X_train)
    y_pred = fit_ridge(X_train, y_train, X_test, alpha=1.0)
    # try glmnet instead
    #from glmnet import ElasticNet
    #model = ElasticNet(alpha=1.0, lambda_path=None, max_iter=1000)
    #model.fit(X_train, y_train)
    #y_pred = model.predict(X_test)

    results = pd.DataFrame({"test":y_test,"pred":y_pred})
    results['diff'] = abs(results.test - results.pred)

    rmse = math.sqrt(mean_squared_error(y_test, y_pred))
    print(f'Ridge Regression RMSE: {rmse}')


Ridge Regression RMSE: 0.1548933544961226
Ridge Regression RMSE: 0.1333650156378639
Ridge Regression RMSE: 0.12702042351844514
Ridge Regression RMSE: 0.1483177284887179
Ridge Regression RMSE: 0.12361475414048702
Ridge Regression RMSE: 0.16493628686329043
Ridge Regression RMSE: 0.14494739432700043
Ridge Regression RMSE: 0.1353771621119425
Ridge Regression RMSE: 0.15885572001061607
Ridge Regression RMSE: 0.135405005410596


In [43]:
train.columns

Index(['PID', 'MS_SubClass', 'MS_Zoning', 'Lot_Frontage', 'Lot_Area', 'Street',
       'Alley', 'Lot_Shape', 'Land_Contour', 'Utilities', 'Lot_Config',
       'Land_Slope', 'Neighborhood', 'Condition_1', 'Condition_2', 'Bldg_Type',
       'House_Style', 'Overall_Qual', 'Overall_Cond', 'Year_Built',
       'Year_Remod_Add', 'Roof_Style', 'Roof_Matl', 'Exterior_1st',
       'Exterior_2nd', 'Mas_Vnr_Type', 'Mas_Vnr_Area', 'Exter_Qual',
       'Exter_Cond', 'Foundation', 'Bsmt_Qual', 'Bsmt_Cond', 'Bsmt_Exposure',
       'BsmtFin_Type_1', 'BsmtFin_SF_1', 'BsmtFin_Type_2', 'BsmtFin_SF_2',
       'Bsmt_Unf_SF', 'Total_Bsmt_SF', 'Heating', 'Heating_QC', 'Central_Air',
       'Electrical', 'First_Flr_SF', 'Second_Flr_SF', 'Low_Qual_Fin_SF',
       'Gr_Liv_Area', 'Bsmt_Full_Bath', 'Bsmt_Half_Bath', 'Full_Bath',
       'Half_Bath', 'Bedroom_AbvGr', 'Kitchen_AbvGr', 'Kitchen_Qual',
       'TotRms_AbvGrd', 'Functional', 'Fireplaces', 'Fireplace_Qu',
       'Garage_Type', 'Garage_Yr_Blt', 'Garage_Fi